In [5]:
import numpy as np
import pandas as pd
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets


from tqdm import tqdm

# Problem 1
Assume that $X_1, X_2, ...$ is a sequence of iid Bernoulli random variables, and $X_i$ has probability $p$ of success.  Assume that $Y_1,Y_2, ...$ is another sequence of iid Bernoulli random variables, and every $Y_i$ has probability $q$ of success. Prove that if $p > q$, then 

$$Prob(\frac{\sum_{i=1}^n X_i}{n} > \frac{1}{2}) > Prob(\frac{\sum_{i=1}^n Y_i}{n}  > \frac{1}{2})$$ 

for all $n$. Hint: write out a binomial expansion of the probabilities involved compare, term by term.

Let $X_i$ be distributed bernoulli with parameter $p_i$ (q) in the problem and $Y_i$ be distributed similarly with parameter $p_i+\delta_i$ which is p above. So we are actually trying to show the reverse, but that's how it was done in office hours.

We then define $X = \sum_{i=1}^n X_i$ with $Y = \sum_{i=1}^n Y_i$. 

We first define a new variable $Z_i \sim Bern(\frac{\delta_i}{1-p_i})$ and we notice that with $Y_i = min(1, X_i + Z_i) we recover $Y_i \sim Bern(p_i + \delta_i)$. Since 

$$(1-p_i)\frac{1-p_i\delta_i}{1-p_i} = 1 - p_i - \delta_i$$

is the probability of drawing $Z_i$ to be zero, so the probability that $Z_i$ is one is given as $p_i + \delta_i$ as desired. 

Then we notice that since $Y_i \geq X_i$ that $\{Y \leq K\} \subset \{X \leq K \}$ we have

$$
P(Y \leq K) \leq P(X \leq K) \\
1 - P(Y > K ) \leq 1 - P(X > K) \\
P(Y > K) \geq P(X > K) \\
P(\sum_{i=1}^n Y_i > \frac{n}{2}) \geq P(\sum_{i=1}^n X_i > \frac{n}{2}) \\
P(\frac{\sum_{i=1}^n Y_i}{n} > \frac{1}{2}) \geq P(\frac{\sum_{i=1}^n X_i}{n} > \frac{1}{2}) \\
$$



# Problem 2
Assume that $X_i$ are iid Bernoulli random variables with probability $p ≥ \frac{2}{3}$ of success.  Use Cramer's theorem to give a lower bound on the number $n$ needed to give 95% confidence that $\sum_{i=1}^n \frac{X_i}{n} > \frac{1}{2}$. 

Recall that cramer's theorem is given as follows

$$
P\left( \left| \frac{\sum_i^n X_i}{n} - p \right| > \epsilon \right) < 2e^{-2 \epsilon^2n}
$$

Using the derivation in class, we know that this implies

$$
P \left ( \frac{\sum_i^n X_i}{n} > \frac{1}{2} \right ) \geq 1 - 2 e^{-2 a^2 n}
$$
where $\frac{1}{2} + a = \frac{2}{3} \implies a = \frac{1}{6}$


Therefore, we need to find for what $n$  

$$
1 - 2 e^{-2 \frac{1}{6}^2 n} = .95
$$

$$
.025 = e^{-2 \frac{1}{6}^2 n}
$$

$$
-18 log(.025) = n
$$

$$
n \approx 67
$$


In [4]:
-18*np.log(.025)

66.399830174050848

# Problem 3
Use scikit learn's random forest classifier to predict survival for the titanic dataset (use an 80-20 train-test split). Experiment with `n_estimators` in range$(20,201,20)$ and `max_depth` in range$(2,10)$ and compare training time and prediction accuracy.  Also compare to the results you obtained last time with a singe tree.

In [8]:
# Import dataset.
df = pd.read_csv("titanic.csv")

# Clean Data
df.drop(['Body'],axis=1, inplace=True)
df.drop(['Boat'],axis=1, inplace=True)
df.drop(['home.dest'],axis=1, inplace=True)
df.drop(['Cabin'],axis=1, inplace=True)
df.dropna(axis=0,inplace=True)

df['Sex'] = df['Sex'].astype("category").cat.codes
df['Embarked'] = df['Embarked'].astype("category").cat.codes


# Test train split.
train_x, test_x, train_y, test_y = train_test_split(df[['Pclass','Sex','Age', 'Fare']], df['Survived'], train_size=.8, test_size=0.2)

# Experiment with max_depth and min_samples_leaf.
base_info = {"accuracy":0, "time":0, "max_depth":0, "min_samples_leaf":0, "method": "none"}
meta_data = []

# grid search
for n_estimators in range(20,201,20):
    for max_depth in range(2,10):
        clf = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators)
        train_d = base_info.copy()
        test_d = base_info.copy()

        # save train meta data
        start = time.time()
        clf.fit(train_x, train_y)
        train_d['time'] = time.time() - start
        train_d['max_depth'] = max_depth
        train_d['n_estimators'] = n_estimators
        train_d['method'] = 'train'
        
        # save test meta data
        start = time.time()
        acc = clf.score(test_x, test_y)
        test_d['accuracy'] = acc
        test_d['time'] = time.time() - start
        test_d['max_depth'] = max_depth
        test_d['n_estimators'] = n_estimators
        test_d['method'] = 'test'
        meta_data.append(train_d)
        meta_data.append(test_d)
        
# print results of min time and max accuracy across methods
best_train_time = min([i for i in meta_data if i['method'] == 'train'], key=lambda x:x['time'])
print("\nbest train time:",
      "".join(["Time: "+ str(best_train_time['time']) +"\n",
               "Max Depth: "+ str(best_train_time['max_depth']) +"\n",
               "n_estimators: "+ str(best_train_time['n_estimators'])])
      , sep="\n")

best_test_time = min([i for i in meta_data if i['method'] == 'test'], key=lambda x:x['time'])
print("\nbest test time:",
      "".join(["Time: "+ str(best_test_time['time']) +"\n",
               "Max Depth: "+ str(best_test_time['max_depth']) +"\n",
               "n_estimators: "+ str(best_test_time['n_estimators']) + "\n",
               "Accuracy: "+ str(best_test_time['accuracy'])])
      , sep="\n")

best_test_accuracy = max([i for i in meta_data if i['method'] == 'test'], key=lambda x:x['accuracy'])
print("\nbest test accuracy:",
      "".join(["Time: "+ str(best_test_accuracy['time']) +"\n",
               "Max Depth: "+ str(best_test_accuracy['max_depth']) +"\n",
               "n_estimators: "+ str(best_test_accuracy['n_estimators']) + "\n",
               "Accuracy: "+ str(best_test_accuracy['accuracy'])])
      , sep="\n")


best_3_depth = max([i for i in meta_data if i['method'] == 'test' and i['max_depth'] == 3], key=lambda x:x['accuracy'])
print("\nbest 3 depth accuracy:",
      "".join(["Time: "+ str(best_3_depth['time']) +"\n",
               "Max Depth: "+ str(best_3_depth['max_depth']) +"\n",
               "n_estimators: "+ str(best_3_depth['n_estimators']) + "\n",
               "Accuracy: "+ str(best_3_depth['accuracy'])])
      , sep="\n")


best train time:
Time: 0.02476191520690918
Max Depth: 3
n_estimators: 20

best test time:
Time: 0.0021190643310546875
Max Depth: 2
n_estimators: 20
Accuracy: 0.746411483254

best test accuracy:
Time: 0.0036940574645996094
Max Depth: 4
n_estimators: 40
Accuracy: 0.813397129187

best 3 depth accuracy:
Time: 0.01976776123046875
Max Depth: 3
n_estimators: 200
Accuracy: 0.789473684211


## Single Tree Output

    best train time:
    Time: 0.0004961490631103516
    Max Depth: 2
    Min Samples Leaf: 71

    best test time:
    Time: 0.00032591819763183594
    Max Depth: 2
    Min Samples Leaf: 61
    Accuracy: 0.789473684211

    best test accuracy:
    Time: 0.00033164024353027344
    Max Depth: 3
    Min Samples Leaf: 1
    Accuracy: 0.808612440191

    best 3 depth accuracy:
    Time: 0.00033164024353027344
    Max Depth: 3
    Min Samples Leaf: 1
    Accuracy: 0.808612440191

As was expected, the random forest was much more accurate on this dataset. However, it is interesting to note that the random forests were much slower than the single tree. Which, again, is what was expected. It still is interesting to see.

# Problem 4
Do the same thing as #3, but on a large dataset related to your final project. 

In [9]:
# Import dataset.
df = pd.read_pickle("recipe_data")

# Test train split.
train_x, test_x, train_y, test_y = train_test_split(df[['calories','fat', 'protein', 'sodium']], df['rating_floor'], train_size=.8, test_size=0.2)

# Experiment with max_depth and min_samples_leaf.
base_info = {"accuracy":0, "time":0, "max_depth":0, "min_samples_leaf":0, "method": "none"}
meta_data = []

#grid search
for n_estimators in range(20,201,20):
    for max_depth in range(2,10):
        clf = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators)
        train_d = base_info.copy()
        test_d = base_info.copy()

        # save train meta data
        start = time.time()
        clf.fit(train_x, train_y)
        train_d['time'] = time.time() - start
        train_d['max_depth'] = max_depth
        train_d['n_estimators'] = n_estimators
        train_d['method'] = 'train'
        
        # save test meta data
        start = time.time()
        acc = clf.score(test_x, test_y)
        test_d['accuracy'] = acc
        test_d['time'] = time.time() - start
        test_d['max_depth'] = max_depth
        test_d['n_estimators'] = n_estimators
        test_d['method'] = 'test'
        meta_data.append(train_d)
        meta_data.append(test_d)
        
# once again, this is just a convoluted way to get the visualization while saving data so it's logged well
best_train_time = min([i for i in meta_data if i['method'] == 'train'], key=lambda x:x['time'])
print("\nbest train time:",
      "".join(["Time: "+ str(best_train_time['time']) +"\n",
               "Max Depth: "+ str(best_train_time['max_depth']) +"\n",
               "n_estimators: "+ str(best_train_time['n_estimators'])])
      , sep="\n")

best_test_time = min([i for i in meta_data if i['method'] == 'test'], key=lambda x:x['time'])
print("\nbest test time:",
      "".join(["Time: "+ str(best_test_time['time']) +"\n",
               "Max Depth: "+ str(best_test_time['max_depth']) +"\n",
               "n_estimators: "+ str(best_test_time['n_estimators']) + "\n",
               "Accuracy: "+ str(best_test_time['accuracy'])])
      , sep="\n")

best_test_accuracy = max([i for i in meta_data if i['method'] == 'test'], key=lambda x:x['accuracy'])
print("\nbest test accuracy:",
      "".join(["Time: "+ str(best_test_accuracy['time']) +"\n",
               "Max Depth: "+ str(best_test_accuracy['max_depth']) +"\n",
               "n_estimators: "+ str(best_test_accuracy['n_estimators']) + "\n",
               "Accuracy: "+ str(best_test_accuracy['accuracy'])])
      , sep="\n")


best_3_depth = max([i for i in meta_data if i['method'] == 'test' and i['max_depth'] == 3], key=lambda x:x['accuracy'])
print("\nbest 3 depth accuracy:",
      "".join(["Time: "+ str(best_3_depth['time']) +"\n",
               "Max Depth: "+ str(best_3_depth['max_depth']) +"\n",
               "n_estimators: "+ str(best_3_depth['n_estimators']) + "\n",
               "Accuracy: "+ str(best_3_depth['accuracy'])])
      , sep="\n")


best train time:
Time: 0.06557917594909668
Max Depth: 2
n_estimators: 20

best test time:
Time: 0.004752159118652344
Max Depth: 2
n_estimators: 20
Accuracy: 0.412346842601

best test accuracy:
Time: 0.024680376052856445
Max Depth: 9
n_estimators: 80
Accuracy: 0.453345900094

best 3 depth accuracy:
Time: 0.005567073822021484
Max Depth: 3
n_estimators: 20
Accuracy: 0.419886899152


## Single Tree on Data Set

    best train time:
    Time: 0.003938198089599609
    Max Depth: 2
    Min Samples Leaf: 61

    best test time:
    Time: 0.0004010200500488281
    Max Depth: 2
    Min Samples Leaf: 51
    Accuracy: 0.419886899152

    best test accuracy:
    Time: 0.0007512569427490234
    Max Depth: 16
    Min Samples Leaf: 1
    Accuracy: 0.434024505184

    best 3 depth accuracy:
    Time: 0.00045299530029296875
    Max Depth: 3
    Min Samples Leaf: 71
    Accuracy: 0.420358152686

Again we see that the random forests had better accuracy ,but much slower train and test times. This is interesting, it would be good in a production system after training was complete, but it is definitely dependent on compute resources ahead of time and may not be excellent for a system that constantly needed to be retrained